In [1]:
import numpy as np
import pandas as pd
import os
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.svm import LinearSVR, SVR, SVC,LinearSVC
from sklearn.datasets import make_regression
from sklearn.preprocessing import normalize 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from utils import dataset_imputer, get_score

In [2]:
# Read in data
df_training_features = pd.read_csv('train_features.csv')
df_training_labels = pd.read_csv('train_labels.csv')
all_pids = [pid for pid in df_training_features['pid'].unique()]

pids_train, pids_val = train_test_split(all_pids, test_size = 0.1)

In [3]:
# def getPatientData(trainingData, pids, patients=0, mode='pid'):
#     if mode == 'number':
#         pids = all_pids[:patients]
#     if len(pids) == 0:
#         return trainingData
#     #pids = np.array(pids).astype(np.float)
#     patients = [trainingData.iloc[idx] for idx in range(0, len(trainingData)) if trainingData['pid'][idx] in pids]    
#     #patientTrainingDataIndex = [trainingData.iloc[idx] for idx, col in enumerate(trainingData) if trainingData['pid'][idx] in pids]    
#     return pd.DataFrame(patients)

# def partitionData(trainingDataPids, trainingPartition=80):
#     validationPartition = 100 - trainingPartition
#     countTraining = int((trainingPartition/100)*len(trainingDataPids))
#     training = trainingDataPids[:countTraining]
#     validation = trainingDataPids[countTraining:]
#     print('')
#     print('Training size: ' + str(countTraining))
#     print('Validation size: ' + str(len(validation)))
#     return training, validation

# def populateData(X,Y):
#     Z = pd.merge(X, Y, on='pid')
#     YData = Z[Y.columns].iloc[:,1:]
#     XData = Z[X.columns].iloc[:,1:]
#     return XData, YData
# import sklearn.metrics as metrics

# TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
#          'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
#          'LABEL_Bilirubin_direct', 'LABEL_EtCO2']

# VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

# #def get_score(df_true, df_submission):
#     df_submission = df_submission.sort_values('pid')
#     df_true = df_true.sort_values('pid')
    
#     #task1 = np.mean([metrics.roc_auc_score(df_true[entry], df_submission[entry]) for entry in TESTS])
#     #task2 = metrics.roc_auc_score(df_true['LABEL_Sepsis'], df_submission['LABEL_Sepsis'])
#     task3 = np.mean([0.5 + 0.5 * np.maximum(0, metrics.r2_score(df_true[entry], df_submission[entry])) for entry in VITALS])
#     #score = np.mean([task1, task2, task3])
#     return task3

In [4]:
X_pid_train = dataset_imputer(df_training_features, method='mean', pid_list=pids_train, fillna=True)
X_pid_val = dataset_imputer(df_training_features, method='mean', pid_list=pids_val, fillna=True)

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
100.0 % - completed

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
100.0 % - completed



In [5]:
Y_pid_train = dataset_imputer(df_training_labels, method=None, pid_list=pids_train, fillna=True)
Y_pid_val = dataset_imputer(df_training_labels, method=None, pid_list=pids_val, fillna=True)

100.0 % - completed

100.0 % - completed



In [6]:
#x_train = X_pid_train[['pid', 'PTT', 'HCO3', 'BaseExcess', 'PaCO2', 'FiO2', 'SaO2','Chloride', 'Hct', 'pH']]
x_train, x_val = X_pid_train.copy(), X_pid_val.copy()
y_train, y_val = Y_pid_train.copy(), Y_pid_val.copy()

feature_columns = ['pid','Age','Temp','RRate','Heartrate','ABPm','SpO2']

x_train, x_val = x_train[feature_columns], x_val[feature_columns]

label_columns = ['pid', 'LABEL_RRate',  'LABEL_ABPm',  'LABEL_SpO2', 'LABEL_Heartrate']

y_train, y_val = y_train[label_columns], y_val[label_columns]

print(x_train.head())
print('*'*100)
print(y_train.head())

   pid   Age       Temp      RRate  Heartrate        ABPm        SpO2
0  1.0  34.0  36.750000  17.000000  77.083333   68.333333  100.000000
0  2.0  86.0  36.000000  18.000000  59.000000   94.636364   96.000000
0  4.0  66.0  36.666667  14.636364  72.545455   80.909091   99.272727
0  6.0  66.0  37.166667  15.833333  87.333333   65.750000   99.333333
0  8.0  42.0  36.000000  17.181818  81.181818  143.900000   97.800000
****************************************************************************************************
       pid  LABEL_RRate  LABEL_ABPm  LABEL_SpO2  LABEL_Heartrate
0        1         12.1        85.4       100.0             59.9
6622     2         20.4        99.1        95.4             65.8
15008    4         17.8        78.8        97.4             71.8
16335    6         17.9        75.1        97.3             80.7
17676    8         18.7       112.8        97.0             92.6


In [7]:
print(x_train.shape)
print(y_train.shape)

(17095, 7)
(17095, 5)


In [8]:
# Train MLPClassifier
#regr =   KNeighborsClassifier(3)
regr = MLPRegressor(alpha=1e-4, hidden_layer_sizes=(200,200,200), random_state=1, solver='adam', max_iter=200)
#regr = RidgeClassifierCV()
#regr = RandomForestClassifier()
regr.fit(x_train.iloc[:,1:], y_train.iloc[:,1:])
print(regr.loss_)

20.71226311778017


In [9]:
print(x_train.iloc[:20,:])
print(y_train.iloc[:20,:])

    pid   Age       Temp      RRate   Heartrate        ABPm        SpO2
0   1.0  34.0  36.750000  17.000000   77.083333   68.333333  100.000000
0   2.0  86.0  36.000000  18.000000   59.000000   94.636364   96.000000
0   4.0  66.0  36.666667  14.636364   72.545455   80.909091   99.272727
0   6.0  66.0  37.166667  15.833333   87.333333   65.750000   99.333333
0   8.0  42.0  36.000000  17.181818   81.181818  143.900000   97.800000
0  10.0  71.0  36.000000  18.090909   78.818182  101.727273   98.000000
0  13.0  73.0  37.600000  16.583333   82.833333   79.916667   99.833333
0  14.0  37.0  36.666667  18.333333   85.285714   82.833333   94.666667
0  20.0  77.0  37.000000  32.444444  115.454545   91.181818   98.000000
0  23.0  82.0   0.000000  12.583333   82.583333   85.500000  100.000000
0  24.0  85.0  36.083333  11.333333   80.833333   66.250000   99.181818
0  26.0  82.0  37.100000  12.666667   81.100000   79.200000   97.500000
0  27.0  58.0  37.333333  20.333333   98.333333   94.916667   95

In [10]:
import sklearn.metrics as metrics

In [13]:

f = regr.predict(x_val.iloc[:,1:])
f = pd.DataFrame(f)
f.columns = list(y_val.columns[1:])
f['pid'] = x_val.iloc[:,0].reset_index(drop=True)

print(get_score(y_val, f, tasks=['task3']))

# VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

# print(np.mean([0.5 + 0.5 * np.maximum(0, metrics.r2_score(y_val[entry], f[entry])) for entry in VITALS]))

0.7015760375674231
(array([       nan,        nan, 0.70157604]), 0.7015760375674231)
